In [1]:
### APO Production MD testing ###

In [2]:

from SBLMDCOVDOCK import SBLSettings

settings = SBLSettings.GROMACS_Settings()

import pandas as pd
import os

import gmxapi as gmx
import subprocess
import matplotlib.pyplot as plt
#collect APO structures 

# AMPC - https://journals.asm.org/doi/10.1128/AAC.02073-20
ampc = "6T3D"
# KPC-2 - BLDB: http://dx.doi.org/10.1021/ACS.JMEDCHEM.7B00158
kpc2 = "5UL8"
# OXA-10 - BLDB: https://www.pnas.org/doi/full/10.1073/pnas.241442898
oxa10 = "1K55"

structures = pd.DataFrame({"PDBID": [ampc, kpc2, oxa10], 
                           "Name": ["AmpC", "KPC2", "OXA10"]})


In [3]:
def plot_xvg(filename):
    # Read the file and get lines
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Extract labels and units from the file
    x_label = [line.split('"')[1] for line in lines if '@    xaxis  label' in line][0]
    y_label = [line.split('"')[1] for line in lines if '@    yaxis  label' in line][0]
    title = filename.split("/")[-1].replace(".xvg", "") + " " + [line.split('"')[1] for line in lines if '@    title' in line][0]
    # Read the data, skipping comment lines starting with '#', '@', or '&'
    data = pd.read_csv(filename, comment='@', skiprows=24, delim_whitespace=True, names=['x', 'y'], header=None)

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(data['x'], data['y'])
    plt.title(title)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.grid(True)
    plt.show()

# Call the function with the provided file



In [4]:
#Perform free production md of the apo protein

for pdbcode in structures.PDBID:
    pdbcode = structures.PDBID[1]
    print(pdbcode)
    break
    md_mdp = os.path.join(settings.config, 'md.mdp')
    topo_path = os.path.join(settings.structures_output, "APO_" + pdbcode+ ".top")

    input_path = os.path.join(settings.structures_output, "APO_" + pdbcode + "_npt.gro")
    tpr_path = os.path.join(settings.structures_output, "APO_" + pdbcode + "_md.tpr")

    grompp_command = ["gmx", "grompp", "-f", md_mdp, "-c", input_path, "-p", topo_path, "-o", tpr_path, "-r", input_path, "-v"]
    subprocess.run(grompp_command, check=True)

    mdrun_command = ["gmx", "mdrun", "-v", "-deffnm", tpr_path.replace(".tpr",""), "-nb", "gpu", "-pme", "gpu" ]
    subprocess.run(mdrun_command, check=True)
    # break
    # grompp_command = ["gmx", "grompp", "-f", md_mdp, "-c", input_path, "-p", topo_path, "-o", tpr_path.replace(".tpr", "_nbcpu.tpr"), "-r", input_path]
    # subprocess.run(grompp_command, check=True)

    # mdrun_command = ["gmx", "mdrun", "-deffnm", tpr_path.replace(".tpr","_nbcpu"), "-nb", "cpu"]
    # subprocess.run(mdrun_command, check=True)


    # grompp_command = ["gmx", "grompp", "-f", md_mdp, "-c", input_path, "-p", topo_path, "-o", tpr_path.replace(".tpr", "_nbgpu.tpr"), "-r", input_path]
    # subprocess.run(grompp_command, check=True)

    # mdrun_command = ["gmx", "mdrun", "-deffnm", tpr_path.replace(".tpr","_nbgpu"), "-nb", "gpu"]
    # subprocess.run(mdrun_command, check=True)


    print("Done with " + pdbcode)

5UL8


In [4]:
for pdbcode in structures.PDBID:
    
    md_mdp = os.path.join(settings.config, 'md.mdp')
    topo_path = os.path.join(settings.structures_output, "APO_" + pdbcode+ ".top")

    xtc_path = os.path.join(settings.structures_output, "APO_" + pdbcode + "_md.xtc")
    tpr_path = os.path.join(settings.structures_output, "APO_" + pdbcode + "_md.tpr")
    xtc_out = xtc_path.replace(".xtc", "_noPBC.xtc")

    trjconv_command = ['gmx', 'trjconv', 
                       '-f', xtc_path, 
                       '-s', tpr_path,  
                       "-o", xtc_out,
                        '-pbc', 'mol',
                        "-center"]
    
    subprocess.run(trjconv_command, input=b"1\n0\n", check=True)

    trjconv_command = ['gmx', 'trjconv', 
                       '-f', xtc_path, 
                       '-s', tpr_path,  
                       "-o", xtc_out.replace(".xtc", ".pdb"),
                        '-pbc', 'mol',
                        "-center"]
    
    subprocess.run(trjconv_command, input=b"1\n1\n", check=True)


                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/SBL_MD_CovDock
Command line:
  gmx trjconv -f prod_structures/APO/APO_6T3D_md.xtc -s prod_structures/APO/APO_6T3D_md.tpr -o prod_structures/APO/APO_6T3D_md_noPBC.xtc -pbc mol -center

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file prod_structures/APO/APO_6T3D_md.tpr, VERSION 2023.2 (single precision)
Reading file prod_structures/APO/APO_6T3D_md.tpr, VERSION 2023.2 (single precision)
Group     0 (         System) has 66524 elements
Group     1 (        Protein) has  5581 elements
Group     2 (      Protein-H) has  2800 elements
Group     3 (        C-alpha) has   358 elements
Group     4 (       Backbone) has  1074 elements
Group     5 (      MainChain) has  1433 elements
Group     6 (   MainChain+Cb) has  1762 elements
Group     7 (    MainChain+H) has  1767 elements
Group     8 (      Si

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'


Last frame        100 time 1000.000    ->  frame     99 time  990.000      
 ->  frame    100 time 1000.000      
Last written: frame    100 time 1000.000


GROMACS reminds you: "The Microsecond is Within Reach" (P.J. Van Maaren)

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/SBL_MD_CovDock
Command line:
  gmx trjconv -f prod_structures/APO/APO_5UL8_md.xtc -s prod_structures/APO/APO_5UL8_md.tpr -o prod_structures/APO/APO_5UL8_md_noPBC.xtc -pbc mol -center

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file prod_structures/APO/APO_5UL8_md.tpr, VERSION 2023.2 (single precision)
Reading file prod_structures/APO/APO_5UL8_md.tpr, VERSION 2023.2 (single precision)
Group     0 (         System) has 55080 elements
Group     1 (        Protein) has  4034 elements
Group     2 (      Protein-H) has  2026 elements
Group     3 (        C-alpha) has  

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 1: 'Protein'
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'
Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 1: 'Protein'


Last frame          5 time   50.000    ->  frame      4 time   40.000      
 ->  frame      5 time   50.000      
Last written: frame      5 time   50.000


GROMACS reminds you: "The Microsecond is Within Reach" (P.J. Van Maaren)

                     :-) GROMACS - gmx trjconv, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/alexi/Documents/SBL_MD_CovDock
Command line:
  gmx trjconv -f prod_structures/APO/APO_1K55_md.xtc -s prod_structures/APO/APO_1K55_md.tpr -o prod_structures/APO/APO_1K55_md_noPBC.xtc -pbc mol -center

Will write xtc: Compressed trajectory (portable xdr format): xtc
Reading file prod_structures/APO/APO_1K55_md.tpr, VERSION 2023.2 (single precision)
Reading file prod_structures/APO/APO_1K55_md.tpr, VERSION 2023.2 (single precision)
Group     0 (         System) has 47383 elements
Group     1 (        Protein) has  3869 elements
Group     2 (      Protein-H) has  1939 elements
Group     3 (        C-alpha) has  

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 0: 'System'


Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 1: 'Protein'
Select group for output
Selected 1: 'Protein'


Last frame        100 time 1000.000    ->  frame     99 time  990.000      
 ->  frame    100 time 1000.000      
Last written: frame    100 time 1000.000


GROMACS reminds you: "The Microsecond is Within Reach" (P.J. Van Maaren)



In [ ]:
#Perform free production md of the apo protein - testing for GPUs etc

for pdbcode in structures.PDBID:
    md_mdp = os.path.join(settings.config, 'md.mdp')
    topo_path = os.path.join(settings.structures_output, "APO_" + pdbcode+ ".top")

    input_path = os.path.join(settings.structures_output, "APO_" + pdbcode + "_npt.gro")
    tpr_path = os.path.join(settings.structures_output, "APO_" + pdbcode + "_md.tpr")

    grompp_command = ["gmx", "grompp", "-f", md_mdp, "-c", input_path, "-p", topo_path, "-o", tpr_path, "-r", input_path]
    subprocess.run(grompp_command, check=True)

    mdrun_command = ["gmx", "mdrun", "-deffnm", tpr_path.replace(".tpr","")]
    subprocess.run(mdrun_command, check=True)

    grompp_command = ["gmx", "grompp", "-f", md_mdp, "-c", input_path, "-p", topo_path, "-o", tpr_path.replace(".tpr", "_nbcpu.tpr"), "-r", input_path]
    subprocess.run(grompp_command, check=True)

    mdrun_command = ["gmx", "mdrun", "-deffnm", tpr_path.replace(".tpr","_nbcpu"), "-nb", "cpu"]
    subprocess.run(mdrun_command, check=True)


    grompp_command = ["gmx", "grompp", "-f", md_mdp, "-c", input_path, "-p", topo_path, "-o", tpr_path.replace(".tpr", "_nbgpu.tpr"), "-r", input_path]
    subprocess.run(grompp_command, check=True)

    mdrun_command = ["gmx", "mdrun", "-deffnm", tpr_path.replace(".tpr","_nbgpu"), "-nb", "gpu"]
    subprocess.run(mdrun_command, check=True)


    print("Done with " + pdbcode)